In [5]:
import pandas as pd
import requests
import time

# 🔑 카카오 REST API 키 입력
REST_API_KEY = "YOUR_REST_API_KEY"

def geocode(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {"개인 API여서 비공개하겠음"}"}
    params = {"query": address}

    res = requests.get(url, headers=headers, params=params).json()

    if res.get('documents'):
        doc = res['documents'][0]
        address_info = doc.get('road_address') or doc.get('address')
        if address_info:
            x = address_info['x']  # 경도
            y = address_info['y']  # 위도
            return float(y), float(x)
    return None, None

df = pd.read_excel(r"C:\Users\KimGW\Desktop\중복 주소 제거.xlsx")


address_col = "주소" 

# 위경도 컬럼 추가
df["lat"] = None
df["lng"] = None


for i, addr in enumerate(df[address_col]):
    if pd.isna(addr):
        continue
    print(f"{i+1}/{len(df)} → {addr}")
    
    lat, lng = geocode(addr)
    df.at[i, "lat"] = lat
    df.at[i, "lng"] = lng

    time.sleep(0.2)


df.to_excel("output_with_latlng4.xlsx", index=False)

1/8597 → 서울특별시 강서구 화곡동 1159
2/8597 → 서울특별시 서대문구 대현동 144
3/8597 → 서울특별시 동작구 사당동 169-32
4/8597 → 서울특별시 은평구 역촌동 43-4
5/8597 → 서울특별시 도봉구 방학동 734
6/8597 → 서울특별시 영등포구 대림동 762-1
7/8597 → 서울특별시 관악구 봉천동 1698-1
8/8597 → 서울특별시 구로구 구로동 107-4
9/8597 → 서울특별시 금천구 시흥동 1013
10/8597 → 서울특별시 서초구 방배동 763-12
11/8597 → 서울특별시 종로구 숭인동 181-121
12/8597 → 서울특별시 노원구 상계동 730-2
13/8597 → 서울특별시 영등포구 신길동 4942
14/8597 → 서울특별시 서초구 신원동 619
15/8597 → 서울특별시 송파구 잠실동 101-1
16/8597 → 서울특별시 도봉구 방학동 508
17/8597 → 서울특별시 노원구 상계동 624
18/8597 → 서울특별시 중랑구 신내동 795
19/8597 → 서울특별시 성북구 돈암동 524
20/8597 → 서울특별시 송파구 신천동 7
21/8597 → 서울특별시 강동구 길동 415-16
22/8597 → 서울특별시 성북구 성북동1가 35-22
23/8597 → 서울특별시 은평구 대조동 4-6
24/8597 → 서울특별시 마포구 창전동 443
25/8597 → 서울특별시 강남구 역삼동 709-5
26/8597 → 서울특별시 마포구 합정동 468
27/8597 → 서울특별시 도봉구 창동 38
28/8597 → 서울특별시 용산구 문배동 24-4
29/8597 → 서울특별시 은평구 신사동 155-1
30/8597 → 서울특별시 구로구 구로동 771-2
31/8597 → 서울특별시 강동구 천호동 22-9
32/8597 → 서울특별시 동대문구 장안동 384-1
33/8597 → 서울특별시 서초구 방배동 998
34/8597 → 서울특별시 강남구 수서동 559
35/8597 → 서울특별시 

In [ ]:
import pandas as pd
import requests
import time

# 🔑 카카오 REST API 키 입력
REST_API_KEY = "개인 API여서 비공개하겠음"

def geocode(address, retries=3, timeout=5):
    """
    주소 → 위경도 변환
    retries: 실패 시 재시도 횟수
    timeout: 요청 제한 시간
    """
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {REST_API_KEY}"}
    params = {"query": address}

    for attempt in range(1, retries + 1):
        try:
            res = requests.get(url, headers=headers, params=params, timeout=timeout)
            res.raise_for_status()
            data = res.json()

            if data.get('documents'):
                doc = data['documents'][0]
                address_info = doc.get('road_address') or doc.get('address')
                if address_info:
                    x = address_info['x']  # 경도
                    y = address_info['y']  # 위도
                    return float(y), float(x)
            return None, None
        except requests.exceptions.RequestException as e:
            print(f"시도 {attempt} 실패: {address}, 오류: {e}")
            time.sleep(1)  # 재시도 전 잠시 대기
    return None, None  # 재시도 모두 실패 시

# 📌 엑셀 파일 로드
df = pd.read_excel(r"C:\Users\KimGW\Desktop\중복 주소 제거.xlsx")

# 📌 주소 컬럼 이름 확인
address_col = "주소"

# 위경도 컬럼 없으면 추가
if "lat" not in df.columns:
    df["lat"] = None
if "lng" not in df.columns:
    df["lng"] = None

# 📌 중단된 곳부터 이어서 처리
for i, addr in enumerate(df[address_col]):
    if pd.isna(addr):
        continue

    # 이미 위경도가 있다면 건너뛰기
    if pd.notna(df.at[i, "lat"]) and pd.notna(df.at[i, "lng"]):
        print(f"{i+1}/{len(df)} → 이미 처리됨, 건너뜀")
        continue

    print(f"{i+1}/{len(df)} → {addr}")
    lat, lng = geocode(addr)
    df.at[i, "lat"] = lat
    df.at[i, "lng"] = lng

    time.sleep(0.2)  # API 호출 제한 방지

# 📌 결과 저장
df.to_excel("output_with_latlng2344.xlsx", index=False)
print("파일 저장 완료: output_with_latlng.xlsx")

In [ ]:
import pandas as pd
import requests
import time

# 🔑 카카오 REST API 키 입력
REST_API_KEY = "YOUR_REST_API_KEY"

def geocode(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {"개인 API여서 비공개하겠음"}"}
    params = {"query": address}

    res = requests.get(url, headers=headers, params=params).json()

    if res.get('documents'):
        doc = res['documents'][0]
        # 일부 주소는 'road_address', 일부는 'address' 만 있음 → 안전하게 처리
        address_info = doc.get('road_address') or doc.get('address')
        if address_info:
            x = address_info['x']  # 경도
            y = address_info['y']  # 위도
            return float(y), float(x)
    return None, None


# 📌 엑셀 파일 로드
df = pd.read_excel(r"C:\Users\KimGW\Desktop\통합 문서1.xlsx")

# 📌 주소 컬럼 이름을 파일에 맞게 수정
address_col = "주소"   # 예: "주소", "address", "addr" 등

# 위경도 컬럼 추가
df["lat"] = None
df["lng"] = None

# 📌 각 주소에 대해 위경도 얻기
for i, addr in enumerate(df[address_col]):
    if pd.isna(addr):
        continue
    print(f"{i+1}/{len(df)} → {addr}")
    
    lat, lng = geocode(addr)
    df.at[i, "lat"] = lat
    df.at[i, "lng"] = lng

    time.sleep(0.2)   # API 호출 제한 방지

# 📌 결과를 새 엑셀로 저장
df.to_excel("output_with_latlng234.xlsx", index=False)

print("파일 저장 완료: output_with_latlng.xlsx")

1/85 → 서울특별시 강남구 수서동 559
2/85 → 서울특별시 강서구 방화동 857
3/85 → 서울특별시 강서구 염창동 275-5
4/85 → 서울특별시 관악구 신림동 1644
5/85 → 서울특별시 광진구 자양동 658-14
6/85 → 서울특별시 도봉구 쌍문동 69-1
7/85 → 서울특별시 노원구 상계동 6013-15
8/85 → 서울특별시 강서구 방화동 182-44
9/85 → 서울특별시 강서구 방화동 863
10/85 → 서울특별시 노원구 공릉동 230
11/85 → 서울특별시 서대문구 북아현동 149
12/85 → 서울특별시 강남구 율현동 8
13/85 → 서울특별시 강서구 방화동 168-38
14/85 → 서울특별시 관악구 신림동 28-9
15/85 → 서울특별시 강서구 방화동 890
16/85 → 서울특별시 서초구 신원동 690
17/85 → 서울특별시 서초구 반포동 1-15
18/85 → 서울특별시 노원구 상계동 109-5
19/85 → 서울특별시 강남구 자곡동 4
20/85 → 서울특별시 강서구 방화동 201-1
21/85 → 서울특별시 동작구 사당동 25-2
22/85 → 서울특별시 강남구 일원동 690-1
23/85 → 서울특별시 강서구 방화동 883
24/85 → 서울특별시 강서구 방화동 859
25/85 → 서울특별시 서초구 잠원동 74-1
26/85 → 서울특별시 성북구 청량리동 949
27/85 → 서울특별시 강서구 방화동 207-5
28/85 → 서울특별시 강서구 방화동 884
29/85 → 서울특별시 강서구 방화동 892
30/85 → 서울특별시 동작구 봉천동 729-24
31/85 → 서울특별시 서초구 잠원동 52-2
32/85 → 서울특별시 강서구 방화동 876
33/85 → 서울특별시 강남구 삼성동 19-4
34/85 → 서울특별시 용산구 효창동 3-197
35/85 → 서울특별시 강서구 방화동 168-114
36/85 → 서울특별시 강서구 방화동 897
37/85 → 서울특별시 서초구 잠원동 74
38/85 → 서

In [12]:
import pandas as pd
import requests
import time

# 🔑 카카오 REST API 키
REST_API_KEY = "YOUR_REST_API_KEY"

def geocode(query):
    """카카오 API로 주소 검색 → 위도/경도 + 전체 주소 반환"""
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {"개인 API여서 비공개하겠음"}"}
    params = {"query": query}

    try:
        response = requests.get(url, headers=headers, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        print(" ❌ API 요청 오류:", e)
        return None, None, None

    if data.get("documents"):
        doc = data["documents"][0]
        addr_info = doc.get("road_address") or doc.get("address")
        if addr_info:
            lng = addr_info["x"]
            lat = addr_info["y"]
            full_addr = addr_info["address_name"]
            return float(lat), float(lng), full_addr
    return None, None, None


# 📌 엑셀 불러오기
df = pd.read_excel(r"C:\Users\KimGW\Desktop\output_with_latlng1.xlsx")

# 📌 컬럼 정의
address_col = "주소"
city_col = "시명"
district_col = "자치구명"
dong_col = "법정동명"
building_col = "건물명"


# 👉 기존 lat/lng 없는 행만 처리
for i, row in df.iterrows():

    # 이미 처리된 행은 건너뛰기
    if pd.notna(row.get("lat")) and pd.notna(row.get("lng")):
        print(f"{i+1}/{len(df)} → 이미 처리됨, SKIP")
        continue

    original_addr = row[address_col]
    print(f"\n▶ {i+1}/{len(df)} — 주소 검색: {original_addr}")

    # 1) 기존 주소로 검색
    lat, lng, new_addr = geocode(original_addr)

    # 2) 기존 주소 실패 → 시명 + 자치구명 + 법정동명 + 건물명으로 검색
    if lat is None or lng is None:
        query2 = f"{row[city_col]} {row[district_col]} {row[dong_col]} {row[building_col]}"
        print(f"   ❗ 1차 실패 → 재검색: {query2}")
        lat, lng, new_addr = geocode(query2)

    # 3) 그래도 실패 → 건너뜀
    if lat is None or lng is None:
        print("   ❌ 최종 실패 → lat/lng 기록 안 함")
        continue

    # 4) 성공 → 값 기록 (주소 컬럼 덮어쓰기)
    df.at[i, address_col] = new_addr
    df.at[i, "lat"] = lat
    df.at[i, "lng"] = lng

    print(f"   ✅ 성공 → 주소 업데이트: {new_addr} / 좌표: ({lat}, {lng})")

    time.sleep(0.2)


# 📌 저장
df.to_excel("output_with_latlng.xlsx", index=False)
print("\n🎉 처리 완료 → output_with_latlng.xlsx 저장됨!")

1/8597 → 이미 처리됨, SKIP
2/8597 → 이미 처리됨, SKIP
3/8597 → 이미 처리됨, SKIP
4/8597 → 이미 처리됨, SKIP
5/8597 → 이미 처리됨, SKIP
6/8597 → 이미 처리됨, SKIP
7/8597 → 이미 처리됨, SKIP
8/8597 → 이미 처리됨, SKIP
9/8597 → 이미 처리됨, SKIP
10/8597 → 이미 처리됨, SKIP
11/8597 → 이미 처리됨, SKIP
12/8597 → 이미 처리됨, SKIP
13/8597 → 이미 처리됨, SKIP
14/8597 → 이미 처리됨, SKIP
15/8597 → 이미 처리됨, SKIP
16/8597 → 이미 처리됨, SKIP
17/8597 → 이미 처리됨, SKIP
18/8597 → 이미 처리됨, SKIP
19/8597 → 이미 처리됨, SKIP
20/8597 → 이미 처리됨, SKIP
21/8597 → 이미 처리됨, SKIP
22/8597 → 이미 처리됨, SKIP
23/8597 → 이미 처리됨, SKIP
24/8597 → 이미 처리됨, SKIP
25/8597 → 이미 처리됨, SKIP
26/8597 → 이미 처리됨, SKIP
27/8597 → 이미 처리됨, SKIP
28/8597 → 이미 처리됨, SKIP
29/8597 → 이미 처리됨, SKIP
30/8597 → 이미 처리됨, SKIP
31/8597 → 이미 처리됨, SKIP
32/8597 → 이미 처리됨, SKIP
33/8597 → 이미 처리됨, SKIP

▶ 34/8597 — 주소 검색: 서울특별시 강남구 수서동 559
   ❗ 1차 실패 → 재검색: 서울특별시 강남구 수서동 강남데시앙포레
   ❌ 최종 실패 → lat/lng 기록 안 함
35/8597 → 이미 처리됨, SKIP
36/8597 → 이미 처리됨, SKIP
37/8597 → 이미 처리됨, SKIP
38/8597 → 이미 처리됨, SKIP
39/8597 → 이미 처리됨, SKIP
40/8597 → 이미 처리됨, SKIP
41/859